In [1]:
!python -m pip install --upgrade pip -qqq -U
!pip install -qqq -U scipy
!pip install huggingface_hub -qqq -U

!pip install -qqq -i https://pypi.org/simple/ bitsandbytes

!pip install git+https://github.com/huggingface/transformers.git -qqq -U #Necessary for merging LoRA adapters onto quantized models.
# !pip install -q -U transformers # if you are facing issues with the dev branch above

!pip install accelerate -qqq -U

!pip install datasets -qqq

# Install peft to allow for LoRA fine-tuning
!pip install -qqq -U peft

In [2]:
from huggingface_hub import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset

In [3]:
cache_dir='.'
hub_model_path = "axel-rda/ARIA-70B-V2-Quantized"

local_model_path = cache_dir + '/' + hub_model_path

In [4]:
repo_path = snapshot_download(
    repo_id=hub_model_path,
    cache_dir=cache_dir,
    local_dir=local_model_path,
    local_dir_use_symlinks=False)

print(f"Repository downloaded to: {local_model_path}")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/331k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

Repository downloaded to: ./axel-rda/ARIA-70B-V2-Quantized


In [5]:
model = AutoModelForCausalLM.from_pretrained(local_model_path)
tokenizer = AutoTokenizer.from_pretrained(local_model_path,use_fast=True,trust_remote_code=True)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [13]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
Tu es un agent de traitement textuel. A partir d'une offre d'emploi tu dois structurer et normaliser les informations de salaire au format JSON selon les règles suivantes :
- Pour un salaire indiqué comme minimum (ex. 'à partir de 30k'), définis-le comme 'montant_min'.
- Si indiqué comme maximum (ex. 'jusqu'à 70000'), définis-le comme 'montant_max'.
- En cas de montant unique, définis-le dans 'montant_min' et 'montant_max'.
- Convertis tous les montants en format décimal y compris ceux exprimés en abréviations. Par exemple, "25k" ou "25ke" devient 25000.0.
- En cas de fourchette de salaire (ex. 'entre 40k et 55ke/an'), segmente-la dans 'montant_min' (40000.0) et 'montant_max' (55000.0) respectivement.
- 'frequence_versement' accepte des valeurs normalisées comme 'an', 'mois', 'semaine', 'jour', 'heure'.
- 'devise' accepte les symboles standards '$', '€' et '£'.
- En cas de manquement de la fréquence ou devise, suppose leur valeurs.
- Si aucun montant explicites n'est mentionné, remplis tous les champs avec la valeur null.
- Des expressions comme "Selon expérience", "A négocier", "Rémunération attractive, ect ne sont pas des montants explicites. Remplis null pour tous les champs.
- Si plusieurs salaires ou fourchettes différentes sont détaillés, liste les toutes. Chaque salaire doit pouvoir etre entierement décrite dans son dictionnaire.
- Privilégie les fourchettes détaillées aux salaire unique ou approximations. Par exemple, si une meme offre mentionne "salaire environ 35k" et ensuite "salaire entre 30 000 et 40 000", privilégie la seconde formulation.
- N'essaye pas de convertir les montants dans d'autre unités que celle donné dans l'offre.
- Assures toi que les informations extraites sont logiquement cohérentes (ex. 'montant_min' <= 'montant_max').

JSON schema : 

{
    "type" : "object",
    "properties" : {
      "salaire" : {
        "type" : "array",
        "items" : {
          "type" : "object",
          "properties" : {
            "montant_min" : {"type" : ["number", "null"]},
            "montant_max" : {"type" : ["number", "null"]},
            "devise" : {"type" : ["string", "null"]},
            "frequence_versement" : {"type" : ["string", "null"]}
          },
          "required": ["montant_min", "montant_max", "devise", "frequence_versement"]
        }
      }
    },
    "required" : ["salaire"]
  }
  
Commence immédiatement ta réponse en JSON valide après la prochaine offre d'emploi :
"""

SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

def get_prompt(instruction):
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text, temperature=0.01, top_p=0.9):
    torch.cuda.empty_cache()
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 temperature=temperature,
                                 top_p=top_p
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [18]:
torch.cuda.empty_cache()

In [42]:
test_data = load_dataset("axel-rda/data_job_salaries_fr", split='test')

In [51]:
%%time
n = 3
print("*** INPUT *** \n\n", test_data['input'][n])
prompt = test_data['input'][n]

print("\n*** OUTPUT *** \n\n")
generated_text = generate(prompt, temperature=0.01)
parse_text(generated_text)

print("\n*** TRUE OUTPUT *** \n\n")
print(test_data['output'][n], "\n")

*** INPUT *** 

 Description de l'entreprise Terry Soot Management Group (TSMG) est une société de collecte de données de terrain fondée en 2017 en Europe. Nous collectons des données là où l'automatisation n'est pas possible. Nous comptons les points, prenons des photos, réalisons des vidéos, enregistrons l’audio et analysons les zones pour chaque détail requis pour pouvoir prendre des décisions plus éclairées. Nos équipes de collecte de données sur le terrain sont réparties en Europe et en Amérique du Nord, prêtes à accepter de nouveaux défis. Objectif du projet L'objectif du projet est d'aider à collecter des images des rues, des principaux points d'intérêt et des espaces publics dans les pays de l'UE. Le projet est réalisé a l’aide des voitures équipés d'une caméra 3D qui recueille des image de la zone autour du véhicule et stockent des images sur des ordinateurs à l'intérieur du véhicule. Plus tard, ces données seront utilisées pour améliorer l'une des cartes en ligne les plus pop

In [47]:
%%time
n = 1
print("*** INPUT *** \n\n", test_data['input'][n])
prompt = test_data['input'][n]

print("\n*** OUTPUT *** \n\n")
generated_text = generate(prompt, temperature=0.01)
parse_text(generated_text)

print("\n*** TRUE OUTPUT *** \n\n")
print(test_data['output'][n])

*** INPUT *** 
 Nous recrutons pour l'un de nos clients, un lead data engineer en CDI pour un client expert en Data et Intelligence Artificielle. Description de l'entreprise Notre client est une deep tech startup qui développe des solutions et des projets sur-mesure de reconnaissance d'image pour tous types d'entreprises Description du poste et des missions En tant que Lead Data Engineer , et auprès du client, tu auras pour missions de : Analyser les données sources et échanger avec les experts métier afin d'identifier et évaluer des cas d'usage métier Leader une équipe de 2 à 5 data software engineers dans le delivery de la solution à implémenter au quotidien Concevoir et mettre en place des systèmes de données résilients et sécurisés (data warehouse, data lake, systèmes temps-réels) Définir les méthodologies de déploiement et plans de migration Construire et déployer les pipelines de données (ETL et ELT) Assurer la migration des données vers les nouveaux environnements Choisir et met

In [48]:
%%time
n = 3
print("*** INPUT *** \n\n", test_data['input'][n])
prompt = test_data['input'][n]

print("\n*** OUTPUT *** \n\n")
generated_text = generate(prompt, temperature=0.01)
parse_text(generated_text)

print("\n*** TRUE OUTPUT *** \n\n")
print(test_data['output'][n], "\n")

*** INPUT *** 

 InVivo est l'un des premiers groupes européens agricoles avec un CA de près de 10 milliards d'euros et un effectif de plus de 13 salariés, dont environ 10 en France. Implanté dans 38 pays, il regroupe 90 sites industriels. Ce pilier de la souveraineté alimentaire intervient sur toute la chaîne de valeur en s'appuyant sur chacun de ses métiers stratégiques : Agriculture ; Malterie ; Malterie ; Meunerie / Boulangerie / Viennoiserie / Pâtisserie ; Jardinerie et distribution alimentaire ; Négoce international de grains ; Viti-vinicole. Un pôle global transverse de solutions innovantes et digitales complète le dispositif pour accélérer la transformation de ces métiers. Bioline by InVivo est la division agricole du groupe InVivo qui s'inscrit dans la démarche de la 3ème voie de l'agriculture Marque ombrelle mondiale, elle représente une alliance unique de savoir-faire reconnus pour accompagner la performance de la ferme France, européenne et mondiale. Filiale de Bioline by I

In [50]:
%%time
n = 4
print("*** INPUT *** \n\n", test_data['input'][n])
prompt = test_data['input'][n]

print("\n*** OUTPUT *** \n\n")
generated_text = generate(prompt, temperature=0.01)
parse_text(generated_text)

print("\n*** TRUE OUTPUT *** \n\n")
print(test_data['output'][n])

*** INPUT *** 

 Descriptif du poste

microDON est une Entreprise Solidaire d'Utilité Sociale (ESUS) de 40 personnes, filiale du groupe KissKissBankBank & co, qui propose des solutions innovantes pour faciliter l'engagement solidaire.

microDON consolide une équipe produit passionnée et engagée. Notre objectif : "la solidarité est l'affaire de tous" - développer de chouettes produits innovants qui permettent de réinventer la générosité en France :

Comme 2 millions de Français chaque mois dans 10 000 magasins, vous faîtes un don à la caisse de votre magasin via L'ARRONDI en caisse . Un produit grand public qui a permis de reverser 45M d'€ aux associations.

530 000 salariés en France sont concernés par L'ARRONDI sur salaire : la possibilité de donner chaque mois quelques euros à l'association de son choix. Un produit solidaire qui a permis de déclencher plus de 1,8 millions de dons

❤️ Mis en place auprès d'une centaine d'entreprises, le don de temps est un produit en constante évoluti